In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import os.path
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import Birch
import  glob
from pprint import pprint

In [5]:
Doc = collections.namedtuple('Doc',['investigator', 'amount', 'abstract'])

def load2(fn):
    doc = nltk.data.load(fn)
    wrds = nltk.tokenize.wordpunct_tokenize(doc)
    investigator = " ".join(wrds[wrds.index("Investigator")+2:
                          min(wrds.index('@' if '@' in wrds else "Abstract", 
                                          wrds.index("Investigator"))-1, 
                              wrds.index('(', wrds.index("Investigator")))])
    amount = int(wrds[wrds.index("Amt") + 4: wrds.index('(', wrds.index("Amt"))][0])
    abstract = " ".join(wrds[wrds.index('Abstract')+2:])
    return Doc(investigator, amount, abstract)



In [7]:
filenames = [fn for fn in glob.iglob('./text/Part*/*/*/*.txt', recursive=False)]
words = []
for i in range(4000):
    words += load2(filenames[i]).abstract.split(" ")

In [260]:
w_vocab = np.unique(words)

In [12]:
vocabulary_size = 30000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
#del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:20])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 1851], ('the', 39472), ('of', 37005), ('.', 29906), ('and', 26950), (',', 25266), ('to', 17952), ('in', 15637), ('will', 10950), ('a', 10623), ('-', 10106), ('The', 8042), ('is', 7897), ('be', 7580), ('for', 7342), ('on', 5197), ('(', 4685), ('are', 4335), ('that', 4277), (')', 4235)]
Sample data [50, 1017, 1842, 5, 169, 5740, 2, 255, 8594, 142] ['In', 'tropical', 'forests', ',', 'most', 'colonization', 'of', 'small', 'disturbed', 'areas']


In [14]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(data, batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(data, batch_size=8, num_skips=2, skip_window=3)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 200
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 4       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 70    # Number of negative examples to sample.

470 ., -> 567 g
470 ., -> 3 .
20058 treefall -> 567 g
20058 treefall -> 19 )
3628 gaps -> 12 is
3628 gaps -> 19 )
19 ) -> 470 .,
19 ) -> 17763 dormant


In [15]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 20001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(data,
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
                
        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print("Average loss at step ", step, ": ", average_loss)
          average_loss = 0
    final_embeddings = normalized_embeddings.eval()
    

Average loss at step  0 :  265.254150391
Average loss at step  2000 :  92.4890298867
Average loss at step  4000 :  33.3492318027
Average loss at step  6000 :  19.0819047289
Average loss at step  8000 :  13.4565618474
Average loss at step  10000 :  10.9229915547
Average loss at step  12000 :  9.80759984016
Average loss at step  14000 :  7.92964790916
Average loss at step  16000 :  6.10592630756
Average loss at step  18000 :  5.88555697155
Average loss at step  20000 :  5.76747018445


In [256]:
from functools import partial

class Infix(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __ror__(self, other):
        return Infix(partial(self.func, other))
    def __call__(self, v1, v2):
        return self.func(v1, v2)

In [16]:
embd = np.array([final_embeddings[datum] for datum in data])

fit = MiniBatchKMeans(n_clusters=5000, max_iter=200000, batch_size=1000)

labels = fit.fit(embd).labels_

/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=3000 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)


In [29]:
gr_dict = {}
rev_gr_dict = {}
for index in range(len(data)):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if reverse_dictionary[data[index]] not in gr_dict:
        gr_dict[reverse_dictionary[data[index]]] = labels[index]
    rev_gr_dict[labels[index]].update([reverse_dictionary[data[index]]])

In [44]:
fit = MiniBatchKMeans(n_clusters=2000, max_iter=200000, batch_size=1000)

In [46]:
vocabulary_size = len(rev_gr_dict)

In [48]:
embd = np.array([final_embeddings[datum[1]] - \
                 final_embeddings[datum[0]] for datum in nl_words])

In [49]:
fit = fit.fit(embd)

In [277]:
fit.set_params(n_clusters=1600)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=200000, max_no_improvement=10,
        n_clusters=1600, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)

In [278]:
nl_words = [(labels[i], labels[i+1]) for i in range(len(labels)-1)]

In [279]:
embd_l = np.array([final_embeddings[datum] for datum in nnl_data])

In [280]:
fit = fit.fit(embd_l[1:] - embd_l[:-1])

In [281]:
labels = fit.labels_ + vocabulary_size

In [282]:
vocabulary_size += fit.n_clusters

In [283]:
for index in range(len(nl_words[:-1])):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if nl_words[index] not in gr_dict:
        gr_dict[nl_words[index]] = labels[index]
    rev_gr_dict[labels[index]].update([nl_words[index]])
    
    

In [284]:
nl_data = [nnl_data[0]]

for i in range(len(nl_words)):
    nl_data.extend([labels[i], nnl_data[i+1]])

nnl_data = labels

In [285]:
for i in range(1):
    graph = tf.Graph()

    with graph.as_default():

      # Input data.
      train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
      train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
      valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

      # Ops and variables pinned to the CPU because of missing GPU implementation
      with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

      # Compute the average NCE loss for the batch.
      # tf.nce_loss automatically draws a new sample of the negative labels each
      # time we evaluate the loss.
      loss = tf.reduce_mean(
          tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                         num_sampled, vocabulary_size))

      # Construct the SGD optimizer using a learning rate of 1.0.
      optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

      # Compute the cosine similarity between minibatch examples and all embeddings.
      norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
      normalized_embeddings = embeddings / norm
      valid_embeddings = tf.nn.embedding_lookup(
          normalized_embeddings, valid_dataset)
      similarity = tf.matmul(
          valid_embeddings, normalized_embeddings, transpose_b=True)

      # Add variable initializer.
      init = tf.initialize_all_variables()

    # Step 5: Begin training.
    num_steps = 20001

    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        init.run()

        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = generate_batch(nl_data,
                batch_size, num_skips, skip_window)
            feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
              if step > 0:
                average_loss /= 2000
              # The average loss is an estimate of the loss over the last 2000 batches.
              print("Average loss at step ", step, ": ", average_loss)
              average_loss = 0
        final_embeddings = normalized_embeddings.eval()


Average loss at step  0 :  268.239318848
Average loss at step  2000 :  44.4170894086
Average loss at step  4000 :  4.91700512797
Average loss at step  6000 :  2.94302289337
Average loss at step  8000 :  2.64875911403
Average loss at step  10000 :  2.46706008929
Average loss at step  12000 :  2.39057153022
Average loss at step  14000 :  2.30778747791
Average loss at step  16000 :  2.23004532826
Average loss at step  18000 :  2.16408869284
Average loss at step  20000 :  2.12751751754


In [286]:
test = collections.Counter(nnl_data).most_common(1)[0][0]

samp = sample(test, rev_gr_dict)
for pre, _, node in anytree.RenderTree(samp):
    print("%s%s" % (pre, node.name))

In [301]:
from anytree.dotexport import RenderTreeGraph

tree = RenderTreeGraph(samp)

tree.to_picture('./test10.png')

In [288]:
import anytree
def sample(label, top, parent=None):
    
    if not(type(label) == str) and label > 3719:
        node = anytree.Node(label, parent=parent)
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        choices = list(top[label].keys())
        indices = [i for i in range(len(choices))]
        choice = choices[np.random.choice(indices, p=probabilities)]
        sample(choice[0], top, node)
        sample(choice[1], top, node)
    else:
        parent = anytree.Node(label, parent=parent)
        probabilities = list(top[label].values())
        if len(probabilities) > 1:
            total = sum(list(top[label].values()))
            probabilities = np.array(probabilities)/total
            choices = list(top[label].keys())
            indices = [i for i in range(len(choices))]
            choice = choices[np.random.choice(indices, p=probabilities)]
            node = anytree.Node(choice, parent=parent)
        else:
            node = anytree.Node(list(top[label].keys())[0], parent=parent)
    
    return node

In [119]:
def sample_sent(label, top):
    if type(list(top[label].keys())[0]) == tuple:
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        choices = list(top[label].keys())
        indices = [i for i in range(len(choices))]
        choice = choices[np.random.choice(indices, p=probabilities)]
        l_sent = sample_sent(choice[0], top)
        r_sent = sample_sent(choice[1], top)
        
        return l_sent +  r_sent
    else:
        return [label]

In [608]:
revdic = open('rev_dic.pkl', 'wb')

pickle.dump(rev_gr_dict, revdic, protocol=3)

revdic.close()
dic = open('dic.pkl', 'wb')
pickle.dump(gr_dict, dic, protocol=3)

np.save('./fembd.npy', final_embeddings)